In [1]:
import pandas as pd
import pprint
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import math
from pylab import rcParams
rcParams['figure.figsize'] = 25, 13

In [2]:
from pymongo import MongoClient

from IPython.core.display import display, HTML
[display(HTML("<style>.container { width:100% !important; }</style>"))];

In [3]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["nyc_crashes"]
    crashes = database.crashes
    crashes_by_year = database.crashes_by_year

In [160]:
def yearly(Year):
    data = crashes_by_year.find_one({"_id": Year})['by_month']
    
    x_values = data.keys()
    y_values_injured = []
    y_values_killed = []
    for month in data:
        injured = 0
        killed = 0
        injured = data[month]['pedestrians']['injured'] + data[month]['cyclists']['injured'] + data[month]['motorists']['injured']
        killed = data[month]['pedestrians']['killed'] + data[month]['cyclists']['killed'] + data[month]['motorists']['killed']
        y_values_injured += [injured]
        y_values_killed += [killed]
    
    show_graph_injury(x_values, y_values_injured, y_values_killed)
    return

In [161]:
def yearly_sum(Year):
    data = crashes_by_year.find_one({"_id": Year})['by_month']
    
    x_values = data.keys()
    y_values_pedestrians = []
    y_values_cyclists = []
    y_values_motorists = []
    for month in data:
        pedestrians = 0
        cyclists = 0
        motorists = 0
        pedestrians = data[month]['pedestrians']['sum']
        cyclists = data[month]['cyclists']['sum']
        motorists = data[month]['motorists']['sum']
        y_values_pedestrians += [pedestrians]
        y_values_cyclists += [cyclists]
        y_values_motorists += [motorists]
    
    show_graph_type(x_values, y_values_pedestrians, y_values_cyclists, y_values_motorists)
    return

In [162]:
def show_graph_injury(x_values, y_values_injured, y_values_killed):
    plt.figure(figsize=(20,10))

    plt.bar(x_values, y_values_killed, color=(0.62,0.07,0.04))
    plt.bar(x_values, y_values_injured, bottom=y_values_killed, color=(0.80,0.52,0.00))
    plt.show()

In [257]:
def show_graph_type(x_values, y_values_pedestrians, y_values_cyclists, y_values_motorists):
    plt.figure(figsize=(20,10))
    
    p1 = plt.bar(x_values, y_values_motorists, color=(0.80,0.50,0.20))
    p2 = plt.bar(x_values, y_values_cyclists, bottom=y_values_motorists, color=(0.75,0.75,0.75))
    bottom_gold = [a+b for a,b in zip(y_values_cyclists, y_values_motorists)]
    p3 = plt.bar(x_values, y_values_pedestrians, bottom=bottom_gold, color=(1.00,0.84,0.00))
    
    plt.ylabel('Anzahl Unfaelle')
    plt.title('Unfaelle pro Monat nach Art')
    plt.legend((p3[0], p2[0], p1[0]), ('Pedestrians', 'Cyclists', 'Motorists'))
    plt.show()

In [258]:
interact(yearly_sum, Year=[data['_id'] for data in crashes_by_year.find().sort('_id', -1)]);

interactive(children=(Dropdown(description='Year', options=('2020', '2019', '2018', '2016', '2013'), value='20…

In [245]:
interact(yearly, Year=[data['_id'] for data in crashes_by_year.find().sort('_id', -1)]);

interactive(children=(Dropdown(description='Year', options=('2020', '2019', '2018', '2016', '2013'), value='20…

In [143]:
year_var
def monthly(Month):
    global year_var
    data = crashes_by_year.find_one({"_id": str(year_var)})['by_month'][str(Month)]
    
    
    #show_graph(x_values, y_values_injured, y_values_killed)
    return data

In [144]:
def out2(Year):
    global year_var
    year_var = int(Year)
    interact(monthly, Month=list(range(1, 13)));
    #return interact(out3, Month=(1,12));
    #return interact(monthly, Month=widgets.IntSlider(min=1,max=12,step=1,value=1))
    return

In [145]:
interact(out2, Year=[data['_id'] for data in crashes_by_year.find().sort('_id', -1)]);

interactive(children=(Dropdown(description='Year', options=('2020', '2019', '2018', '2016', '2013'), value='20…

In [ ]:
from datetime import datetime

start_date = datetime(2018, 4, 24)
end_date = datetime(2018, 5, 24)

dates = pd.date_range(start_date, end_date, freq='B')

options = [(date.strftime('%b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

selection_range_slider

In [ ]:
import datetime
dates = [datetime.date(2015, i, 1) for i in range(1, 13)]
options = [(i.strftime('%b'), i) for i in dates]
widgets.SelectionRangeSlider(
    options=options,
    index=(0, 11),
    description='2015',
    disabled=False
)